In [38]:
## Após o registro de todas máquinas, lembrar de verificar in loco o OS Type e o WL Capability
## para máquinas que possuem Windows

## Robô de Registro de Equipamentos no Sistema de Gestão de Ativos de TI (CMS)

In [39]:
### Importar bibliotecas necessárias do selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

### Importar bibliotecas para o Python
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### Importar bibliotecas para acesso às interfaces
import pyautogui as pag

In [40]:
### Processo de Login com CMS ###

## Abre janela para digitar usuário e senha
usuario = pag.prompt('Digita seu usuário do CMS')
senha = pag.prompt('Digita sua senha do CMS')

## criar o navegador com o webdriver do Chrome
navegador = webdriver.Chrome()

## Link de acesso do PITS da empresa ZF
navegador.get("http://pits-cms.emea.zf-world.com/")

## Preencher as informações de Login e senha
navegador.find_element(By.XPATH, '//*[@id="zid"]').send_keys(usuario)
navegador.find_element(By.XPATH, '//*[@id="password"]').send_keys(senha)

## Clicar no botão para realizar o LOGIN
navegador.find_element(By.XPATH, '//*[@id="submit_login"]').click()

In [41]:
time.sleep(4)

In [42]:
## Fazer a leitura da lista de máquinas que serão registradas no CMS
dfMaquinas = pd.read_excel('equipments.xlsx')

In [43]:
## Verificar as 5 primeiras linhas do dataset para avaliar se os dados estão da forma que precisamos
dfMaquinas.head()

,Celula,CodLP1,DeviceName,Inventario,descricaomaquina,Fabricante,Serie,BU,TIPO,FABRICANTE,MODELO,SO,WLNecessary,WLInstalled,OnCMS
0,AFT,159464,159464 - Embutimento de Pino e Concha 00,425-2,Embutimento de Pino e Concha,Lemförder,274.380-831.52,CC,CLP,SIEMENS,S7-300,EMBEDDED,NO,NO,NO
1,AFT,159467,159467 - Dispositivo de Montagem do AnelTtenso...,2168-2,Dispositivo de Montagem do AnelTtensor -,Allware,8071068,CC,CLP,SIEMENS,S7-200,EMBEDDED,NO,NO,NO
2,C010,173546,173546 - Alimentador automático de pinos 00,685885-2,Alimentador automático de pinos,Schindler,NaN,CC,CLP,SIEMENS,S7-300,EMBEDDED,NO,NO,NO
3,C020,159512,159512 - Furadeira Overdrill 00,732-2,Furadeira Overdrill,Overdrill,31616,CC,CLP,SIEMENS,S7-200,EMBEDDED,NO,NO,NO
4,C020,173545,173545 - Alimentador automático de pinos 00,645884-2,Alimentador automático de pinos,Schindler,NaN,CC,CLP,SIEMENS,S7-300,EMBEDDED,NO,NO,NO


### Loop for para percorrer todos itens da lista de equipamentos e cadastrá-los no sistema

In [44]:
## Define um loop que tenha o tamanho da planilha carregada
pos = 0
tamanhoPlaniha = len(dfMaquinas)
wait = WebDriverWait(navegador, 10)

## Alguns tempos foram inseridos devido a instabilidade da conexão então necessário aguardar carregamento da página
for i in dfMaquinas.CodLP1:
    
    ## Acessa a página de registro de equipamentos
    time.sleep(1)
    navegador.get("http://pits-cms.emea.zf-world.com/items.php?akce=edit_items&deletePost=true&id_schema=104")
    time.sleep(5)
    
    ## Digitar o número do Inventário no txtbox de Inventário
    fastrack = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[1]/td[2]/input')))
    fastrack.send_keys(str(dfMaquinas.Inventario[pos]))
    time.sleep(1)
    
    ## Digitar o número do DeviceName no txtbox DeviceName
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[1]/td[1]/input').send_keys(str(dfMaquinas.DeviceName[pos]))
    time.sleep(1)
    
    ## Clicar para abrir a lista de Category do item
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/a').click()
    time.sleep(2)
    
    ## Verifica na planilha se é CPU, IHM, CNC ou CLP e preenche a informação necessária
    ## Alguns comandos foram feitos utilizando o PYautoGUI
    if (dfMaquinas.TIPO[pos] == "CPU"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("PC")
        time.sleep(1)
        pag.press('down')
        time.sleep(1)
        pag.press('down')
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    elif (dfMaquinas.TIPO[pos] == "IHM"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("HMI")
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    elif (dfMaquinas.TIPO[pos] == "CNC"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("Rack")
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    elif (dfMaquinas.TIPO[pos] == "CLP"):
        if (dfMaquinas.MODELO[pos] == "S7-300"):
            navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("S7-300")
            time.sleep(1)
            pag.press('enter')
            time.sleep(1)
        elif (dfMaquinas.MODELO[pos] == "S7-1200"):
            navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("S7-1200")
            time.sleep(1)
            pag.press('enter')
            time.sleep(1)
        else:
            navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[2]/td/div/div/div/input').send_keys("PLC")
            time.sleep(1)
            pag.press('enter')
            time.sleep(1)
   
    ## Preenche o campo Type detail no txtbox Type detail
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[3]/td/input').send_keys(str(dfMaquinas.MODELO[pos]))
    time.sleep(1)
    
    ## Preenche o campo Fabricante no txtbox Producer
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[4]/td[1]/input').send_keys(str(dfMaquinas.FABRICANTE[pos]))
    time.sleep(1)
    
    ## Preenche a BU, cc ou tc
    if(dfMaquinas.BU[pos] == "CC"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[6]/td[2]/select').click()
        time.sleep(1)
        pag.press('c')
        time.sleep(1)
        pag.press('c')
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    elif(dfMaquinas.BU[pos] == "TC"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[1]/tbody/tr[6]/td[2]/select').click()
        time.sleep(1)
        pag.press('t')
        time.sleep(1)
        pag.press('t')
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    
    ## Preenche o campo Date acquisition
    navegador.find_element(By.XPATH, '//*[@id="date_acquisition"]').send_keys("01.01.2015")
    time.sleep(1)
    
    ## Clicar no campo de OS Type
    if (dfMaquinas.TIPO[pos] != "CLP"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[3]/tbody/tr[1]/td[1]/div/a/span').click()
        time.sleep(1)
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[3]/tbody/tr[1]/td[1]/div/div/div/input').send_keys("Siemens Proprietary")
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    
    ## Preencher "WL Capability WL is not supported" para todas as máquinas
    if (dfMaquinas.TIPO[pos] != "CLP"):
        navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[3]/tbody/tr[1]/td[2]/select').click()
        time.sleep(1)
        pag.press('W')
        time.sleep(1)
        pag.press('enter')
        time.sleep(1)
    
    ## Colocar qual o status do equipamento
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[4]/tbody/tr[1]/td/select').click()
    time.sleep(1)
    pag.press('A')
    time.sleep(1)
    pag.press('enter')
    time.sleep(1)
    
    ## Clica no botão para Salvar para finalizar o registro
    navegador.find_element(By.XPATH, '//*[@id="rightColumn"]/form/fieldset/table[5]/tbody/tr[2]/td/input[1]').click()
    time.sleep(3)
    
    pos = pos + 1
    if pos > tamanhoPlaniha:
        pag.alert("Finalizado!!")
        break